In [8]:
import os
import sys
import pandas as pd

sys.path.insert(0, r'C:\\Users\\Matt\\Documents\\Development\\VolAdjGIPDashboard\\VolAdjGIPDashboard')
#sys.path.insert(0, '/home/matt/src/VolAdjGIPDashboard/VolAdjGIPDashboard')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "VolAdjGIPDashboard.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
django.setup()

In [2]:
from DataAcquisition.models import QuadForecasts
gdp, cpi = QuadForecasts.fetch_usa_gi_data() # 20191108, both series verified against FRED.
display(gdp.tail(5))
display(cpi.tail(12))

DATE
2018-09-30    18732.720
2018-12-31    18783.548
2019-03-31    18927.281
2019-06-30    19021.860
2019-09-30    19121.112
Freq: Q-DEC, Name: GDPC1, dtype: float64

2017-12-31    246.618667
2018-03-31    248.804000
2018-06-30    251.374333
2018-09-30    252.197000
2018-12-31    252.052000
2019-03-31    252.896667
2019-06-30    255.927667
2019-09-30    256.629333
2019-12-31    257.320000
2020-03-31    259.116667
2020-06-30    260.883333
2020-09-30    261.560000
dtype: float64

In [4]:
gdp_set = QuadForecasts.get_gdp_set(gdp) # issue is probably related to GDPNow. # maybe not; deleted code, still lying to me

                         growth  best_estimate
quarter    date                               
2002-03-31 2001-11-23  0.996986   13397.002000
           2001-11-30  0.996885   13397.002000
           2001-12-07  0.997692   13397.002000
           2001-12-14  0.997541   13397.002000
           2001-12-21  0.998346   13397.002000
...                         ...            ...
2020-03-31 2019-12-06  1.001646   19180.295096
           2019-12-13  1.002044   19193.156469
           2019-12-20  1.004075   19262.076561
           2019-12-27  1.003754   19249.734825
           2020-01-03  1.002962   19233.120363

[2561 rows x 2 columns]


In [5]:
gdp_set.tail(20)

,,growth,best_estimate
quarter,date,,
2015-06-30,2015-07-31,1.005332,17405.669000
2015-09-30,2015-10-30,1.004815,17463.222000
2015-12-31,2016-01-29,1.003036,17468.902000
2016-03-31,2016-04-29,1.001795,17556.839000
2016-06-30,2016-07-29,1.005234,17639.417000
2016-09-30,2016-10-28,1.005554,17735.074000
2016-12-31,2017-01-27,1.005012,17824.231000
2017-03-31,2017-04-28,1.006683,17925.256000
2017-06-30,2017-07-28,1.005185,18021.048000


In [11]:
for forecast in QuadForecasts.objects.all().order_by(('date')):
    print(f"quarter_end_date={forecast.quarter_end_date}, date={forecast.date}, quad={forecast.quad}")

quarter_end_date=2019-03-31, date=2019-01-11, quad=1
quarter_end_date=2019-03-31, date=2019-01-18, quad=1
quarter_end_date=2019-03-31, date=2019-01-25, quad=1
quarter_end_date=2019-03-31, date=2019-02-01, quad=1
quarter_end_date=2019-03-31, date=2019-02-08, quad=1
quarter_end_date=2019-03-31, date=2019-02-15, quad=1
quarter_end_date=2019-03-31, date=2019-02-22, quad=1
quarter_end_date=2019-03-31, date=2019-03-01, quad=1
quarter_end_date=2019-03-31, date=2019-03-08, quad=1
quarter_end_date=2019-03-31, date=2019-03-15, quad=1
quarter_end_date=2019-03-31, date=2019-03-22, quad=1
quarter_end_date=2019-03-31, date=2019-03-29, quad=1
quarter_end_date=2019-06-30, date=2019-04-05, quad=3
quarter_end_date=2019-06-30, date=2019-04-12, quad=3
quarter_end_date=2019-06-30, date=2019-04-19, quad=3
quarter_end_date=2019-06-30, date=2019-04-26, quad=3
quarter_end_date=2019-06-30, date=2019-05-03, quad=3
quarter_end_date=2019-06-30, date=2019-05-10, quad=3
quarter_end_date=2019-06-30, date=2019-05-17, 

In [5]:
gdp, cpi = QuadForecasts.fetch_usa_gi_data()
usa_quads = QuadForecasts.determine_quads(gdp, cpi).dropna()

usa_quads = usa_quads[
    (usa_quads.index.get_level_values('date') <= usa_quads.index.get_level_values('quarter')) &
    (usa_quads.index.get_level_values('date') > usa_quads.index.get_level_values('quarter') - pd.offsets.QuarterEnd())
].droplevel('quarter')

usa_quads['changed'] = usa_quads.quad != usa_quads.shift(1).quad
#usa_quads = usa_quads[usa_quads.changed == True].drop(['changed'], axis='columns')

In [6]:
usa_quads.tail(20)

,gdp_roc,cpi_roc,quad,changed
date,,,,
2019-08-30,-20.498506,-5.388707,4.0,False
2019-09-06,-20.498506,-5.388707,4.0,False
2019-09-13,-20.498506,-5.388707,4.0,False
2019-09-20,-20.498506,-5.388707,4.0,False
2019-09-27,-20.498506,-5.388707,4.0,False
2019-10-04,5.303102,33.255639,2.0,True
2019-10-11,5.303102,33.255639,2.0,False
2019-10-18,-1.003723,33.255639,3.0,True
2019-10-25,-4.287898,33.255639,3.0,False
